In [ ]:
# pip install sparqlwrapper

In [ ]:
import pandas as pd
from pandas.io.json import json_normalize
import sqlite3

import re

import sys
from SPARQLWrapper import SPARQLWrapper, JSON

In [ ]:
def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query().convert()
    return json_normalize(result["results"]["bindings"])

In [ ]:
### Query all diseases with Disease Ontology ID and get their alternative names if available


endpoint_url = "https://query.wikidata.org/sparql"

query = """SELECT ?name ?doidLabel ?nameLabel ?aliasLabel WHERE { 
  ?name wdt:P699 ?doidLabel
        OPTIONAL {
            ?name skos:altLabel ?aliasLabel .
            FILTER (lang(?aliasLabel)='en')
            }
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }
}"""

results = get_results(endpoint_url, query)
results

C:\Users\PC\AppData\Local\Temp\ipykernel_13140\4046432563.py:8: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  return json_normalize(result["results"]["bindings"])


,name.type,name.value,aliasLabel.xml:lang,aliasLabel.type,aliasLabel.value,doidLabel.type,doidLabel.value,nameLabel.xml:lang,nameLabel.type,nameLabel.value
0,uri,http://www.wikidata.org/entity/Q72000,en,literal,Lemierre syndrome,literal,DOID:11337,en,literal,Lemierre's syndrome
1,uri,http://www.wikidata.org/entity/Q72000,en,literal,acute sore throat,literal,DOID:11337,en,literal,Lemierre's syndrome
2,uri,http://www.wikidata.org/entity/Q72000,en,literal,human necrobacillosis,literal,DOID:11337,en,literal,Lemierre's syndrome
3,uri,http://www.wikidata.org/entity/Q72000,en,literal,postanginal sepsis,literal,DOID:11337,en,literal,Lemierre's syndrome
4,uri,http://www.wikidata.org/entity/Q73518,en,literal,non-bacterial thrombotic endocarditis,literal,DOID:0060068,en,literal,marantic endocarditis
...,...,...,...,...,...,...,...,...,...,...
50108,uri,http://www.wikidata.org/entity/Q19001364,en,literal,"EPISODIC ATAXIA, TYPE 7; EA7",literal,DOID:0050995,en,literal,episodic ataxia type 7
50109,uri,http://www.wikidata.org/entity/Q28024560,en,literal,CMD1K,literal,DOID:0110437,en,literal,dilated cardiomyopathy 1K
50110,uri,http://www.wikidata.org/entity/Q28024560,en,literal,dilated cardiomyopathy type 1K,literal,DOID:0110437,en,literal,dilated cardiomyopathy 1K
50111,uri,http://www.wikidata.org/entity/Q28024561,en,literal,"CARDIOMYOPATHY, DILATED, 1JJ",literal,DOID:0110438,en,literal,dilated cardiomyopathy 1JJ


In [ ]:
df_disease = results[["doidLabel.value","nameLabel.value","aliasLabel.value"]]
df_disease = df_disease.rename(columns = lambda col: col.replace("Label.value", ""))
df_disease.head()

,doid,name,alias
0,DOID:11337,Lemierre's syndrome,Lemierre syndrome
1,DOID:11337,Lemierre's syndrome,acute sore throat
2,DOID:11337,Lemierre's syndrome,human necrobacillosis
3,DOID:11337,Lemierre's syndrome,postanginal sepsis
4,DOID:0060068,marantic endocarditis,non-bacterial thrombotic endocarditis


In [ ]:
df_disease = df_disease[(df_disease['doid'].str.contains("DOID"))]
df_disease.sort_values('doid')

,doid,name,alias
48937,DOID:0001816,angiosarcoma,NaN
7341,DOID:0002116,pterygium,Surfer's Eye
16144,DOID:0014667,metabolic disease,metabolic illness
16143,DOID:0014667,metabolic disease,metabolic diseases
16142,DOID:0014667,metabolic disease,disorder of metabolic process
...,...,...,...
1137,DOID:999,eosinophilia,Eosinophil count raised
4225,DOID:9993,hypoglycemia,hypoglycaemia
21067,DOID:9997,peripartum cardiomyopathy,postpartum peripartum cardiomyopathy
21066,DOID:9997,peripartum cardiomyopathy,antepartum peripartum cardiomyopathy


In [ ]:
### Query all diseases with Disease Ontology ID and get their health specialty type ( for identification of infectious disease)


endpoint_url = "https://query.wikidata.org/sparql"

query2 =  """
SELECT ?doidLabel ?diseaseLabel ?specialty_typeLabel WHERE { 
  ?disease wdt:P699 ?doidLabel .
    OPTIONAL { 
        ?disease wdt:P1995 ?specialty_type .
    }  
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }
}
"""

results2 = get_results(endpoint_url, query2)
results2

C:\Users\PC\AppData\Local\Temp\ipykernel_13140\4046432563.py:8: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  return json_normalize(result["results"]["bindings"])


,doidLabel.type,doidLabel.value,diseaseLabel.xml:lang,diseaseLabel.type,diseaseLabel.value,specialty_typeLabel.xml:lang,specialty_typeLabel.type,specialty_typeLabel.value
0,literal,DOID:2377,en,literal,multiple sclerosis,en,literal,neurology
1,literal,DOID:437,en,literal,myasthenia gravis,en,literal,neurology
2,literal,DOID:150,en,literal,mental disorder,en,literal,psychiatry
3,literal,DOID:150,en,literal,mental disorder,en,literal,clinical psychology
4,literal,DOID:14351,en,literal,Riedel's fibrosing thyroiditis,en,literal,endocrinology
...,...,...,...,...,...,...,...,...
11637,literal,DOID:0080045,en,literal,Kniest dysplasia,en,literal,medical genetics
11638,literal,DOID:4321,en,literal,large-cell acanthoma,NaN,NaN,NaN
11639,literal,DOID:0060740,en,literal,methylmalonyl-CoA mutase deficiency,NaN,NaN,NaN
11640,literal,DOID:0111875,en,literal,microphthalmia–dermal aplasia–sclerocornea syn...,NaN,NaN,NaN


In [ ]:
df_infect = results2[["doidLabel.value", "diseaseLabel.value", "specialty_typeLabel.value"]]
df_infect = df_infect.rename(columns = lambda col: col.replace("Label.value", ""))
df_infect.head()

,doid,disease,specialty_type
0,DOID:2377,multiple sclerosis,neurology
1,DOID:437,myasthenia gravis,neurology
2,DOID:150,mental disorder,psychiatry
3,DOID:150,mental disorder,clinical psychology
4,DOID:14351,Riedel's fibrosing thyroiditis,endocrinology


In [ ]:
df_infect['specialty_type'].unique()

array(['neurology', 'psychiatry', 'clinical psychology', 'endocrinology',
       'pulmonology', 'infectious diseases', 'immunology', 'psychology',
       'ophthalmology', 'gastroenterology', 'cardiology', 'optometry',
       'orthopedics', 'pediatrics', 'medical genetics', 'hematology',
       'rheumatology', 'dermatology', 'oncology', 'military medicine',
       nan, 'urology', 'obstetrics', 'psychomotor education',
       'oral surgery', 'gynaecology', 'psychotherapy', 'otolaryngology',
       'angiology', 'nephrology', 'hepatology', 'dentistry',
       'Medicina fisica y rehabilitacion',
       'physical medicine and rehabilitation', 'helminthologist',
       'maternal-fetal medicine', 'neonatology', 'oral medicine',
       'narcology', 'addiction medicine', 'emergency medicine',
       'obstetrics and gynaecology', 'oral and maxillofacial surgery',
       'nutrition', 'gynecologic oncology', 'phlebology', 'neurosurgery',
       'allergology', 'endocrine surgery', 'diabetology',
   

In [ ]:
# create indicator whether disease specialty contains infect
df_infect['infectious']= df_infect["specialty_type"].str.contains(r'infect')*1


In [ ]:
# clean up doid column
df_infect = df_infect[(df_infect['doid'].str.contains("DOID"))]
# for each doid, identify if disease is infectious or not
df_infect = df_infect.sort_values(['doid','infectious'], ascending=False).groupby('doid').head(1)
df_infect

,doid,disease,specialty_type,infectious
9840,DOID:9997,peripartum cardiomyopathy,cardiology,0
6742,DOID:9993,hypoglycemia,endocrinology,0
11154,DOID:999,eosinophilia,hematology,0
9410,DOID:9988,tertiary neurosyphilis,infectious diseases,1
6319,DOID:9987,orbit sarcoma,NaN,NaN
...,...,...,...,...
426,DOID:0040002,salicylate sensitivity,NaN,NaN
5604,DOID:0040001,shrimp allergy,NaN,NaN
4309,DOID:0014667,metabolic disease,endocrinology,0
961,DOID:0002116,pterygium,ophthalmology,0


In [ ]:
# Merge disease alt names with indicator of whether disease is infectious or not
disease_kb = df_disease.merge(df_infect[["doid","infectious"]], how='left', on="doid")
disease_kb

,doid,name,alias,infectious
0,DOID:11337,Lemierre's syndrome,Lemierre syndrome,1
1,DOID:11337,Lemierre's syndrome,acute sore throat,1
2,DOID:11337,Lemierre's syndrome,human necrobacillosis,1
3,DOID:11337,Lemierre's syndrome,postanginal sepsis,1
4,DOID:0060068,marantic endocarditis,non-bacterial thrombotic endocarditis,0
...,...,...,...,...
50072,DOID:0050995,episodic ataxia type 7,"EPISODIC ATAXIA, TYPE 7; EA7",NaN
50073,DOID:0110437,dilated cardiomyopathy 1K,CMD1K,0
50074,DOID:0110437,dilated cardiomyopathy 1K,dilated cardiomyopathy type 1K,0
50075,DOID:0110438,dilated cardiomyopathy 1JJ,"CARDIOMYOPATHY, DILATED, 1JJ",0


In [ ]:
# Create sqlite database and cursor
conn = sqlite3.connect('disease_kb.db')
c = conn.cursor()
# Create the table
c.execute("""CREATE TABLE IF NOT EXISTS disease (
            doid ID,
            name text,
            alias text,
            infectious real
            )""")
conn.commit()

disease_kb.to_sql('disease', conn, if_exists='append', index=False)

test = conn.execute('SELECT * from disease')
print(test)